# Visualization: Trading Session

In [6]:
import pandas as pd
import numpy as np

import altair as alt
import seaborn as sns

### 1. Define parameters and Load model

In [7]:
from trading_bot.agent import Agent

model_name = 'aaapl1mbig_4'
test_stock = 'data/AAPLtest.csv'
window_size = 10
debug = True

agent = Agent(window_size, pretrained=True, model_name=model_name)

### 2. Load test data

### 3. Running Eval

In [8]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data["Close"][1] - test_data["Close"][0]

test_result, history, actionCollection = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-09-17 13:52:52 justin-desktop-ubuntu root[17554] DEBUG switching to TensorFlow for CPU
2020-09-17 13:52:52 justin-desktop-ubuntu root[17554] DEBUG Buy at: $11.05
2020-09-17 13:52:52 justin-desktop-ubuntu root[17554] INFO aaapl1mbig_4: USELESS



Data lerngth 49
Final decision:  HOLD  at  $11.26


In [9]:
len(actionCollection),len(test_data) # <- should be same length (minus one for current pred)

(49, 50)

In [10]:
test_data["realaction"] = np.sign(test_data["Close"].pct_change().shift(-1))
# we use 2 to sell, 1 to buy, 0 to hold
test_data["realaction"][test_data["realaction"]==-1.] = 2.
realActions = test_data["realaction"].values
# the last entry is the next day entry, we need to skip it
realActions = realActions[:-1]
test_data.head()

,High,Low,Close,Volume,realaction
0,11.119617,11.025241,11.051266,3.545151,1.0
1,11.196168,11.097075,11.155034,1.226301,2.0
2,11.136902,11.084690,11.048796,2.557836,2.0
3,11.043066,11.037626,11.004324,2.355639,1.0
4,11.072699,11.050011,11.033972,2.654514,2.0


In [11]:
#for i in range(15):
#    print(realActions[i],actionCollection[i])

In [12]:
# basic implementation
from sklearn.metrics import classification_report
target_names = ['hold', 'buy', 'sell']
cr = classification_report(realActions, actionCollection, target_names=target_names)
print(cr)

              precision    recall  f1-score   support

        hold       0.02      1.00      0.04         1
         buy       1.00      0.04      0.07        28
        sell       0.00      0.00      0.00        20

    accuracy                           0.04        49
   macro avg       0.34      0.35      0.04        49
weighted avg       0.57      0.04      0.04        49



/home/justin/miniconda3/envs/tf115/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 4. Visualize

In [15]:
try:
    df = pd.read_csv(test_stock,index_col="Datetime",parse_dates=["Datetime"])
except ValueError:
    df = pd.read_csv(test_stock,index_col="Date",parse_dates=["Date"])
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50 entries, 2019-01-02 09:31:00 to 2019-01-02 10:20:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    50 non-null     float64
 1   High    50 non-null     float64
 2   Low     50 non-null     float64
 3   Close   50 non-null     float64
 4   Volume  50 non-null     int64  
dtypes: float64(4), int64(1)
memory usage: 2.3 KB


In [16]:
import backtrader as bt

class SmaCross(bt.Strategy):

    params = (
        ('allowshorts',1),
        ('printLog',False)
    )

    def __init__(self):
        self.pos = 0
        self.data_live = False
        self.isShort = False
        

    def next(self):
        for d in self.datas:
            # print(dn,self.crosses[d._name][0])
            pos = self.getposition(d).size
            if self.pos < len(actionCollection):
                action = actionCollection[self.pos]
                if not pos:  # no market / no orders
                    if action == 1:
                        self.buy(data=d)
                        self.isShort = False
                    elif action == 2 and self.p.allowshorts == 1:
                        self.sell(data=d)
                        self.isShort = True
                else:
                    if action == 1 and self.isShort:
                        self.close(data=d)
                        #self.buy(data=d)
                        #self.isShort = False
                    elif action == 2 and not self.isShort:
                        self.close(data=d)
                        #self.sell(data=d)
                        #self.isShort = True
            else:
                print("skip bc end of array")
            self.pos += 1

    def notify_trade(self, trade):
        dt = self.data.datetime.date()
        if trade.isclosed and self.p.printLog:
            print('{} {} Closed: PnL Gross {}, Net {}'.format(
                                                dt,
                                                trade.data._name,
                                                round(trade.pnl,2),
                                                round(trade.pnlcomm,2)))
            
    def notify_data(self, data, status, *args, **kwargs):
        if status == data.LIVE:
            self.data_live = True
            print("yay Live data!")


In [17]:
cerebro = bt.Cerebro(optreturn=True,maxcpus=None,cheat_on_open=True) # cheat on open bc i can instantly buy if signal appears
data = bt.feeds.PandasData(dataname=df)

startcash = 10000   
# Set our desired cash start
cerebro.broker.setcash(startcash)
# for stock in self.stocks:
cerebro.adddata(data)

# strategy
# cerebro.addstrategy(SmaCross, pfast=2,pslow=6) # not used bc opt strategy

cerebro.addstrategy(SmaCross) # https://backtest-rookies.com/2017/06/26/optimize-strategies-backtrader/


# cerebro.optstrategy(SmaCross, pfast=pfast,pslow=pslow) # https://backtest-rookies.com/2017/06/26/optimize-strategies-backtrader/

# add analyzer
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio') #  timeframe=bt.TimeFrame.Months
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown') #  timeframe=bt.TimeFrame.Months
cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns') #  timeframe=bt.TimeFrame.Months
cerebro.addanalyzer(bt.analyzers.SQN, _name='SQN') #  timeframe=bt.TimeFrame.Months


# sizer
cerebro.addsizer(bt.sizers.PercentSizer, percents=98)

results = cerebro.run()
end = cerebro.broker.get_value() - startcash
print(end)

skip bc end of array
48.107479653790506


In [20]:
%matplotlib inline
cerebro.plot(iplot=False)

2020-09-17 14:15:35 justin-desktop-ubuntu matplotlib.pyplot[17554] DEBUG Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2020-09-17 14:15:35 justin-desktop-ubuntu matplotlib.pyplot[17554] DEBUG Loaded backend TkAgg version unknown.


ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/home/justin/miniconda3/envs/tf115/lib/python3.7/site-packages/matplotlib/dates.py)

In [ ]:
def sqn_eval(nr):
    if nr <= 1.9:
        return "below average"
    elif nr <= 2.4:
        return "average"
    elif nr <= 2.9:
        return "good"
    elif nr <= 5.:
        return "excellent"
    elif nr <= 6.9:
        return "superb"
    elif nr > 6.9:
        return "HOLY GRAIL - $$$"
    else:
        return "ERROR"

def single_analytics(results,cerebro,startcash,printout=False):
    #sharpe
    sharper = results[0].analyzers.SharpeRatio.get_analysis()["sharperatio"]

    # drawdown
    maxdrawdown = results[0].analyzers.DrawDown.get_analysis().max.drawdown
    avgdrawdown = results[0].analyzers.DrawDown.get_analysis().drawdown
    # returns
    annualreturnpct = results[0].analyzers.Returns.get_analysis()["rnorm100"]
    monthlyreturnpct = annualreturnpct / 12
    # SQN SystemQualityNumber - https://www.backtrader.com/docu/analyzers-reference/
    sqn, nrtrades = results[0].analyzers.SQN.get_analysis()["sqn"],results[0].analyzers.SQN.get_analysis()["trades"]
    #Print out the final result
    if printout:
        print("\n")
        print('Sharpe Ratio: ', sharper)
        print("Avg Drawdown: %.2f pct, Max Drawdown: %.2f pct"%(avgdrawdown,maxdrawdown))
        print("Yearly return %.2f pct, Monthly return %.2f pct"%(annualreturnpct,monthlyreturnpct))
        print("SQN: %f, nr of trades: %d"%(sqn,nrtrades))
        print("SQN Evaluation: %s"%sqn_eval(sqn))
    return sharper,sqn,nrtrades,avgdrawdown,maxdrawdown,annualreturnpct,monthlyreturnpct

sharper,sqn,nrtrades,avgdrawdown,maxdrawdown,annualreturnpct,monthlyreturnpct = single_analytics(results,cerebro,startcash,printout=True)

# data analysis

## eurusd 1m 10

Sharpe Ratio:  None

Avg Drawdown: 0.15 pct, Max Drawdown: 0.30 pct

Yearly return 66.65 pct, Monthly return 5.55 pct

SQN: 1.155293, nr of trades: 210

SQN Evaluation: below average